# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 3 2022 10:25AM,247721,10,Enova-10740,Emerginnova,Released
1,Oct 3 2022 10:25AM,247721,10,M-6-23993-1,Emerginnova,Released
2,Oct 3 2022 9:34AM,247718,10,8593033,Eywa Pharma Inc.,Released
3,Oct 3 2022 9:34AM,247718,10,8593624,Eywa Pharma Inc.,Released
4,Oct 3 2022 9:34AM,247718,10,8604274,Eywa Pharma Inc.,Released
5,Oct 3 2022 9:34AM,247718,10,8604294,Eywa Pharma Inc.,Released
6,Oct 3 2022 9:34AM,247718,10,8604316,Eywa Pharma Inc.,Released
7,Oct 3 2022 9:34AM,247718,10,8604318,Eywa Pharma Inc.,Released
8,Oct 3 2022 9:29AM,247717,15,8603756,"Brookfield Pharmaceuticals, LLC",Released
9,Oct 3 2022 9:29AM,247717,15,8604269,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,247714,Executing,1
18,247715,Released,18
19,247717,Released,23
20,247718,Released,6
21,247721,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247714,1.0,NaN
247715,NaN,18.0
247717,NaN,23.0
247718,NaN,6.0
247721,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
247613,0.0,1.0
247684,0.0,15.0
247690,0.0,1.0
247692,0.0,1.0
247700,1.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
247613,0,1
247684,0,15
247690,0,1
247692,0,1
247700,1,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,247613,0,1
1,247684,0,15
2,247690,0,1
3,247692,0,1
4,247700,1,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,247613,,1
1,247684,,15
2,247690,,1
3,247692,,1
4,247700,1,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 3 2022 10:25AM,247721,10,Emerginnova
2,Oct 3 2022 9:34AM,247718,10,Eywa Pharma Inc.
8,Oct 3 2022 9:29AM,247717,15,"Brookfield Pharmaceuticals, LLC"
31,Oct 3 2022 9:27AM,247613,15,"Brookfield Pharmaceuticals, LLC"
32,Oct 3 2022 9:21AM,247715,10,ISDIN Corporation
50,Oct 3 2022 9:17AM,247713,10,ISDIN Corporation
106,Oct 3 2022 9:17AM,247714,20,"HVL, LLC dba Atrium Innovations"
107,Oct 3 2022 9:12AM,247711,10,ISDIN Corporation
167,Oct 3 2022 9:00AM,247710,10,ISDIN Corporation
227,Oct 3 2022 8:50AM,247709,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 3 2022 10:25AM,247721,10,Emerginnova,,2
1,Oct 3 2022 9:34AM,247718,10,Eywa Pharma Inc.,,6
2,Oct 3 2022 9:29AM,247717,15,"Brookfield Pharmaceuticals, LLC",,23
3,Oct 3 2022 9:27AM,247613,15,"Brookfield Pharmaceuticals, LLC",,1
4,Oct 3 2022 9:21AM,247715,10,ISDIN Corporation,,18
5,Oct 3 2022 9:17AM,247713,10,ISDIN Corporation,,56
6,Oct 3 2022 9:17AM,247714,20,"HVL, LLC dba Atrium Innovations",1,
7,Oct 3 2022 9:12AM,247711,10,ISDIN Corporation,,60
8,Oct 3 2022 9:00AM,247710,10,ISDIN Corporation,38,22
9,Oct 3 2022 8:50AM,247709,10,Bio-PRF,,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 3 2022 10:25AM,247721,10,Emerginnova,2,
1,Oct 3 2022 9:34AM,247718,10,Eywa Pharma Inc.,6,
2,Oct 3 2022 9:29AM,247717,15,"Brookfield Pharmaceuticals, LLC",23,
3,Oct 3 2022 9:27AM,247613,15,"Brookfield Pharmaceuticals, LLC",1,
4,Oct 3 2022 9:21AM,247715,10,ISDIN Corporation,18,
5,Oct 3 2022 9:17AM,247713,10,ISDIN Corporation,56,
6,Oct 3 2022 9:17AM,247714,20,"HVL, LLC dba Atrium Innovations",,1
7,Oct 3 2022 9:12AM,247711,10,ISDIN Corporation,60,
8,Oct 3 2022 9:00AM,247710,10,ISDIN Corporation,22,38
9,Oct 3 2022 8:50AM,247709,10,Bio-PRF,12,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 3 2022 10:25AM,247721,10,Emerginnova,2,
1,Oct 3 2022 9:34AM,247718,10,Eywa Pharma Inc.,6,
2,Oct 3 2022 9:29AM,247717,15,"Brookfield Pharmaceuticals, LLC",23,
3,Oct 3 2022 9:27AM,247613,15,"Brookfield Pharmaceuticals, LLC",1,
4,Oct 3 2022 9:21AM,247715,10,ISDIN Corporation,18,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 3 2022 10:25AM,247721,10,Emerginnova,2.0,NaN
1,Oct 3 2022 9:34AM,247718,10,Eywa Pharma Inc.,6.0,NaN
2,Oct 3 2022 9:29AM,247717,15,"Brookfield Pharmaceuticals, LLC",23.0,NaN
3,Oct 3 2022 9:27AM,247613,15,"Brookfield Pharmaceuticals, LLC",1.0,NaN
4,Oct 3 2022 9:21AM,247715,10,ISDIN Corporation,18.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 3 2022 10:25AM,247721,10,Emerginnova,2.0,0.0
1,Oct 3 2022 9:34AM,247718,10,Eywa Pharma Inc.,6.0,0.0
2,Oct 3 2022 9:29AM,247717,15,"Brookfield Pharmaceuticals, LLC",23.0,0.0
3,Oct 3 2022 9:27AM,247613,15,"Brookfield Pharmaceuticals, LLC",1.0,0.0
4,Oct 3 2022 9:21AM,247715,10,ISDIN Corporation,18.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3220173,214.0,39.0
15,495330,24.0,0.0
16,743114,3.0,0.0
19,247703,1.0,0.0
20,247714,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3220173,214.0,39.0
1,15,495330,24.0,0.0
2,16,743114,3.0,0.0
3,19,247703,1.0,0.0
4,20,247714,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,214.0,39.0
1,15,24.0,0.0
2,16,3.0,0.0
3,19,1.0,0.0
4,20,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,214.0
1,15,Released,24.0
2,16,Released,3.0
3,19,Released,1.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Executing,39.0,0.0,0.0,0.0,1.0
Released,214.0,24.0,3.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Executing,39.0,0.0,0.0,0.0,1.0
1,Released,214.0,24.0,3.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Executing,39.0,0.0,0.0,0.0,1.0
1,Released,214.0,24.0,3.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()